# Display map with GeoJSON layers

https://ipyleaflet.readthedocs.io/en/latest/api_reference/geo_json.html


## Configuration

In [1]:
# Specify the locality whose OSM file we want to load

locality = 'Mount Eliza Sample'
margin   = 20
zoom     = 14

## Load required modules

In [2]:
import json
import os
import sys

from ipyleaflet import Map, Marker, GeoJSON

import pandas as pd

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.detection_map as detection_map

## Display map of detected points with boundary

In [3]:
# Version of locality strings with underscores
locality_clean  = locality.replace(' ', '_')
locality_margin = '{0:s}_{1:d}m'.format(locality_clean, margin)

In [4]:
# Load boundary of the sample area, so we can draw it on the map
margin_filename = os.path.join(os.pardir, 'data_sources', 'Locality_{0:s}_margin.geojson'.format(locality_clean))
margin_layer, margin_data = detection_map.load_layer(margin_filename)

center = detection_map.get_centroid(margin_data)

m_detect = Map(center=center, zoom=zoom)
m_detect.add_layer(margin_layer)

# Load detected points and add to map

detection_log = os.path.join(os.pardir, 'detections', locality_margin, 'detection_log.csv')
count = detection_map.load_points(m_detect, detection_log)
print('Point count: ' + str(count))

# Display the map
m_detect

Point count: 114


Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## Display map of detected lines with boundary

In [16]:
# Version of locality strings with underscores
locality_clean  = locality.replace(' ', '_')
locality_margin = '{0:s}_{1:d}m'.format(locality_clean, margin)

detected_filename = os.path.join(
    module_path_root,
    'detections',
    locality_margin,
    'hit.geojson'
)

print(detected_filename)

D:\TensorFlow2\TFODCourse\minor_thesis\detections\Mount_Eliza_Sample_20m\hit.geojson


In [15]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(margin_filename)

center = detection_map.get_centroid(margin_data)

# Load lines that we drew from the detected points
detected_layer, detected_data = detection_map.load_layer(detected_filename, color='red')

# Build the map
m_detect2 = Map(center=center, zoom=zoom)
m_detect2.add_layer(margin_layer)
m_detect2.add_layer(detected_layer)

# Display the map
m_detect2

Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## Display map of Open Street Map cycleways

In [ ]:
locality_filename = os.path.join(os.pardir, 'data_sources', 'LGA_boundary_Mount_Eliza.geojson')
osm_filename      = os.path.join(os.pardir, 'data_sources', 'Locality_OSM_cycleway_Mount_Eliza.geojson')

locality_layer, locality_data = detection_map.load_layer(locality_filename, color='orange')
osm_layer,      osm_data      = detection_map.load_layer(osm_filename,      color='green')

center = detection_map.get_centroid(margin_data)

# Build the map
m_detect3 = Map(center=center, zoom=zoom)
m_detect3.add_layer(margin_layer)
m_detect3.add_layer(locality_layer)
m_detect3.add_layer(osm_layer)

# Display the map
m_detect3

OSM is missing a segment of Nepean Highway from Mount Eliza Way to Old Mornington Road.

This corresponds to way id="204762387".  It is NOT tagged with a "cycleway", it is tagged with "bicycle"="yes" and "foot"="yes".  This generally means that bicycle traffic (and foot traffic) is allowed, but not that there is a cycleway.

The small segment that is missing on Humphries Road from Walkers Road to Overport road was clipped in the OSM extract for the locality, it is just outside the official shape of the locality.  If we widen our OSM extract we can see that it is a cycleway.  way id="671976035".

In [ ]:
# locality_filename = os.path.join('data_sources', 'LGA_boundary_' + locality.replace(' ', '_') + '.geojson')
# pbn_existing_filename = os.path.join('data_sources', 'PBN_Existing.geojson')
# pbn_planned_filename = os.path.join('data_sources', 'PBN_Planned.geojson')
# osm_filename = os.path.join('data_sources', 'Locality_OSM_cycleway_' + locality.replace(' ', '_') + '.geojson')
# detect_filename = os.path.join('detections', locality_detect.replace(' ', '_'), 'detected_points.geojson')

## Demonstrate how to use Folium and save to .PNG

In [ ]:
import folium
import json

style_red = {
    'color': 'red',
    'weight': 2,
    'fillColor': 'red',
    'fillOpacity': 0.2
}

with open(margin_filename) as f:
    gj_margin = json.load(f)
    
with open(detected_filename) as f:
    gj_detected = json.load(f)

m = folium.Map(location=center, zoom_start=zoom)
folium.GeoJson(gj_margin, name='boundary').add_to(m)
folium.GeoJson(gj_detected, name='detected', style_function=lambda x:style_red).add_to(m)
m

In [ ]:
# Save folium to PNG, requires selenium, geckodriver, Firefox
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(os.path.join(os.pardir, 'map_detect.png'))

In [ ]:
import folium
import json

style_red = {
    'color': 'red',
    'weight': 2,
    'fillColor': 'red',
    'fillOpacity': 0.2
}

with open(margin_filename) as f:
    gj_margin = json.load(f)
    
with open(osm_filename) as f:
    gj_detected = json.load(f)

m = folium.Map(location=center, zoom_start=zoom)
folium.GeoJson(gj_margin, name='boundary').add_to(m)
folium.GeoJson(gj_detected, name='detected', style_function=lambda x:style_red).add_to(m)
m


In [ ]:
# Save folium to PNG, requires selenium, geckodriver, Firefox
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save(os.path.join(os.pardir, 'map_osm.png'))